In [1]:
import pandas as pd

In [2]:
train = pd.read_csv(r"C:\Users\basde\Downloads\playground-series-s3e24\train.csv")

In [3]:
from sklearn.preprocessing import RobustScaler

In [4]:
train['BMI'] = train['weight(kg)'] / (train['height(cm)'] / 100)**2
train['Chol_LDL'] = (train['LDL'] + train['Cholesterol']) * 2
train = train.drop(columns=['waist(cm)','height(cm)','weight(kg)','LDL', 'Cholesterol'])


In [5]:
train.columns

Index(['id', 'age', 'eyesight(left)', 'eyesight(right)', 'hearing(left)',
       'hearing(right)', 'systolic', 'relaxation', 'fasting blood sugar',
       'triglyceride', 'HDL', 'hemoglobin', 'Urine protein',
       'serum creatinine', 'AST', 'ALT', 'Gtp', 'dental caries', 'smoking',
       'BMI', 'Chol_LDL'],
      dtype='object')

In [6]:
scaler = RobustScaler()
scaled_data = scaler.fit_transform(train)
scaled_df = pd.DataFrame(scaled_data,
                         columns=train.columns)
print(scaled_df.head())

         id       age  eyesight(left)  eyesight(right)  hearing(left)  \
0 -1.000000  1.000000           -1.25            -1.00            0.0   
1 -0.999987  2.000000           -1.00            -0.75            1.0   
2 -0.999975 -1.333333           -1.50            -1.25            0.0   
3 -0.999962 -0.333333            1.25             0.50            0.0   
4 -0.999950 -0.666667            1.25             0.00            0.0   

   hearing(right)  systolic  relaxation  fasting blood sugar  triglyceride  \
0             0.0    0.8750    0.750000            -0.153846      2.102273   
1             1.0    1.5625    0.416667             3.923077     -0.681818   
2             0.0   -0.1875   -0.250000            -1.307692      0.931818   
3             0.0    0.6250    0.833333            -0.384615      1.000000   
4             0.0    0.0000   -0.166667            -0.384615     -0.318182   

   ...  hemoglobin  Urine protein  serum creatinine       AST     ALT  \
0  ...        0.75 

In [7]:
scaled_df['id'] = train['id']
scaled_df['smoking'] = train['smoking']

In [8]:
print(scaled_df.head())

   id       age  eyesight(left)  eyesight(right)  hearing(left)  \
0   0  1.000000           -1.25            -1.00            0.0   
1   1  2.000000           -1.00            -0.75            1.0   
2   2 -1.333333           -1.50            -1.25            0.0   
3   3 -0.333333            1.25             0.50            0.0   
4   4 -0.666667            1.25             0.00            0.0   

   hearing(right)  systolic  relaxation  fasting blood sugar  triglyceride  \
0             0.0    0.8750    0.750000            -0.153846      2.102273   
1             1.0    1.5625    0.416667             3.923077     -0.681818   
2             0.0   -0.1875   -0.250000            -1.307692      0.931818   
3             0.0    0.6250    0.833333            -0.384615      1.000000   
4             0.0    0.0000   -0.166667            -0.384615     -0.318182   

   ...  hemoglobin  Urine protein  serum creatinine       AST     ALT  \
0  ...        0.75            0.0               0.5 -0.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report

In [10]:
X = scaled_df.drop('smoking', axis=1)
y = scaled_df['smoking']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [21]:
logreg = LogisticRegression(solver='liblinear', random_state=42)

In [22]:
rfe_selector = RFE(estimator=logreg, n_features_to_select=5, step=1, verbose=2)
rfe_selector.fit(X_train, y_train)

Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.


,estimator,LogisticRegre...r='liblinear')
,n_features_to_select,5
,step,1
,verbose,2
,importance_getter,'auto'
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1


In [23]:
selected_features_mask = rfe_selector.support_
selected_feature_names = X_train.columns[selected_features_mask].tolist()
print(f"\nSelected Features ({len(selected_feature_names)}):")
print(selected_feature_names)
print("\n--- Feature Ranking (Higher rank = Eliminated Earlier) ---")
feature_ranking = pd.Series(rfe_selector.ranking_, index=X_train.columns)
print(feature_ranking.sort_values(ascending=False)) # Sort descending to see eliminated first


Selected Features (5):
['triglyceride', 'hemoglobin', 'Gtp', 'dental caries', 'Chol_LDL']

--- Feature Ranking (Higher rank = Eliminated Earlier) ---
id                     16
eyesight(left)         15
hearing(left)          14
eyesight(right)        13
relaxation             12
hearing(right)         11
ALT                    10
fasting blood sugar     9
systolic                8
Urine protein           7
AST                     6
serum creatinine        5
BMI                     4
HDL                     3
age                     2
hemoglobin              1
Gtp                     1
dental caries           1
triglyceride            1
Chol_LDL                1
dtype: int32


In [24]:
logreg = LogisticRegression(solver='newton-cg', random_state=42, max_iter=100)

In [25]:
rfe_selector = RFE(estimator=logreg, n_features_to_select=5, step=1, verbose=2)
rfe_selector.fit(X_train, y_train)

Fitting estimator with 20 features.


C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\optimize.py:100: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(


Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.


,estimator,LogisticRegre...r='newton-cg')
,n_features_to_select,5
,step,1
,verbose,2
,importance_getter,'auto'
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1


In [26]:
selected_features_mask = rfe_selector.support_
selected_feature_names = X_train.columns[selected_features_mask].tolist()
print(f"\nSelected Features ({len(selected_feature_names)}):")
print(selected_feature_names)
print("\n--- Feature Ranking (Higher rank = Eliminated Earlier) ---")
feature_ranking = pd.Series(rfe_selector.ranking_, index=X_train.columns)
print(feature_ranking.sort_values(ascending=False)) # Sort descending to see eliminated first


Selected Features (5):
['triglyceride', 'hemoglobin', 'Gtp', 'dental caries', 'Chol_LDL']

--- Feature Ranking (Higher rank = Eliminated Earlier) ---
id                     16
eyesight(left)         15
hearing(left)          14
eyesight(right)        13
relaxation             12
hearing(right)         11
ALT                    10
fasting blood sugar     9
systolic                8
Urine protein           7
AST                     6
serum creatinine        5
BMI                     4
HDL                     3
age                     2
hemoglobin              1
Gtp                     1
dental caries           1
triglyceride            1
Chol_LDL                1
dtype: int32


In [27]:
logreg = LogisticRegression(solver='lbfgs', random_state=42, max_iter=100)

In [28]:
rfe_selector = RFE(estimator=logreg, n_features_to_select=5, step=1, verbose=2)
rfe_selector.fit(X_train, y_train)

Fitting estimator with 20 features.


C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.


,estimator,LogisticRegre...ndom_state=42)
,n_features_to_select,5
,step,1
,verbose,2
,importance_getter,'auto'
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1


In [29]:
selected_features_mask = rfe_selector.support_
selected_feature_names = X_train.columns[selected_features_mask].tolist()
print(f"\nSelected Features ({len(selected_feature_names)}):")
print(selected_feature_names)
print("\n--- Feature Ranking (Higher rank = Eliminated Earlier) ---")
feature_ranking = pd.Series(rfe_selector.ranking_, index=X_train.columns)
print(feature_ranking.sort_values(ascending=False)) # Sort descending to see eliminated first


Selected Features (5):
['triglyceride', 'hemoglobin', 'Gtp', 'dental caries', 'Chol_LDL']

--- Feature Ranking (Higher rank = Eliminated Earlier) ---
id                     16
eyesight(left)         15
eyesight(right)        14
hearing(left)          13
relaxation             12
hearing(right)         11
ALT                    10
fasting blood sugar     9
systolic                8
Urine protein           7
AST                     6
serum creatinine        5
BMI                     4
HDL                     3
age                     2
hemoglobin              1
Gtp                     1
dental caries           1
triglyceride            1
Chol_LDL                1
dtype: int32
